# Bengali NID Intent Classification with SmolLM2-135M

Fine-tune SmolLM2-135M on Bengali NID customer service dataset for intent classification.

**Environment:** Google Colab Pro with L4 GPU (24GB)

| Component | Value |
|-----------|-------|
| Model | SmolLM2-135M (135M params) |
| Dataset | Bengali NID (407 intents, ~78k train, ~11k eval) |
| Method | Generative SFT + LoRA |
| Language | Bengali questions → English intent tags |

## 1. Install Dependencies

In [1]:
!pip install -q transformers>=4.46.0 "datasets>=3.0.0,<4.0.0" trl>=0.12.0 peft>=0.13.0 accelerate>=1.0.0 bitsandbytes>=0.44.0 scikit-learn pandas

## 2. Check GPU & Mount Google Drive

In [2]:
import torch

# Check GPU
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"GPU: {gpu_name}")
    print(f"VRAM: {gpu_memory:.1f} GB")
else:
    print("No GPU available! Go to Runtime > Change runtime type > GPU")
    raise RuntimeError("GPU required")

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create output directory for model
import os
OUTPUT_DIR = "/content/drive/MyDrive/models/smollm2-bengali-nid-intent"
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Model output directory: {OUTPUT_DIR}")

# ============================================================
# DATASET PATH - Upload your CSVs to this folder in Google Drive
# ============================================================
DATASET_DIR = "/content/drive/MyDrive"
os.makedirs(DATASET_DIR, exist_ok=True)

print(f"\n>>> Upload your CSV files to: {DATASET_DIR}")
print("    - sts_train.csv")
print("    - sts_eval.csv")
print("    - tag_answer.csv")
print("\nOr change DATASET_DIR to where your files are located.")

GPU: NVIDIA L4
VRAM: 23.8 GB
Mounted at /content/drive
Model output directory: /content/drive/MyDrive/models/smollm2-bengali-nid-intent

>>> Upload your CSV files to: /content/drive/MyDrive
    - sts_train.csv
    - sts_eval.csv
    - tag_answer.csv

Or change DATASET_DIR to where your files are located.


## 3. Verify Dataset Files

Make sure your CSV files are in Google Drive at the path shown above.

In [3]:
import os

# Check if files exist in Google Drive
train_path = f"{DATASET_DIR}/sts_train.csv"
eval_path = f"{DATASET_DIR}/sts_eval.csv"
tag_path = f"{DATASET_DIR}/tag_answer.csv"

files_status = {
    "sts_train.csv": os.path.exists(train_path),
    "sts_eval.csv": os.path.exists(eval_path),
    "tag_answer.csv": os.path.exists(tag_path),
}

print("Dataset files status:")
for fname, exists in files_status.items():
    status = "✓ Found" if exists else "✗ Missing"
    print(f"  {status}: {fname}")

if not all(files_status.values()):
    missing = [f for f, exists in files_status.items() if not exists]
    print(f"\n❌ Missing files: {missing}")
    print(f"Please upload them to: {DATASET_DIR}")
    raise FileNotFoundError(f"Missing dataset files in {DATASET_DIR}")
else:
    print(f"\n✓ All files found in {DATASET_DIR}")

Dataset files status:
  ✓ Found: sts_train.csv
  ✓ Found: sts_eval.csv
  ✓ Found: tag_answer.csv

✓ All files found in /content/drive/MyDrive


## 4. Load and Analyze Dataset

In [4]:
import pandas as pd
from collections import Counter

# Load CSV files from Google Drive
print("Loading dataset files from Google Drive...")
train_df = pd.read_csv(train_path)
eval_df = pd.read_csv(eval_path)
tag_answer_df = pd.read_csv(tag_path)

print(f"Train samples: {len(train_df)}")
print(f"Eval samples: {len(eval_df)}")
print(f"Unique tags in train: {train_df['tag'].nunique()}")
print(f"Unique tags in eval: {eval_df['tag'].nunique()}")
print(f"Tags with answers: {len(tag_answer_df)}")

# Show sample
print(f"\nSample from training data:")
print(f"  Question: {train_df.iloc[0]['question']}")
print(f"  Tag: {train_df.iloc[0]['tag']}")

Loading dataset files from Google Drive...
Train samples: 78616
Eval samples: 11457
Unique tags in train: 407
Unique tags in eval: 403
Tags with answers: 407

Sample from training data:
  Question: "একাউন্ট লক করা হয়েছে" দেখাচ্ছে, সমাধান কী?
  Tag: account_locked


In [5]:
# Build intent labels from training data
INTENT_TAGS = sorted(train_df['tag'].unique().tolist())
print(f"Total unique intents: {len(INTENT_TAGS)}")

# Create mappings
ID2INTENT = {i: intent for i, intent in enumerate(INTENT_TAGS)}
INTENT2ID = {intent: i for i, intent in enumerate(INTENT_TAGS)}

# Show top 15 tags by frequency
print(f"\nTop 15 tags by frequency:")
tag_counts = train_df['tag'].value_counts()
for tag, count in tag_counts.head(15).items():
    print(f"  {tag}: {count}")

Total unique intents: 407

Top 15 tags by frequency:
  fraction: 494
  permanent_address_change_fees: 381
  spouse_name_correction_new: 231
  parent_spouse_name_correct_or_add_document_new: 229
  parents_name_correction_new: 226
  goodbye: 218
  picture_done_but_lost_or_no_sms_slip: 215
  service_provided: 213
  disability_no_hands_registration_procedure: 206
  abroad_smart_card_collection_return: 206
  reissue_urgent_card_delivery_time: 206
  signature_to_fingerprint_reversal_not_allowed: 206
  reissue_smart_card_download_not_available: 206
  abroad_illegal_resident_nid: 206
  abroad_embassy_walk_in_registration: 206


## 5. Configuration

In [6]:
# ============================================================
# CONFIGURATION
# ============================================================

# Model
MODEL_NAME = "HuggingFaceTB/SmolLM2-135M"

# Data
MAX_SEQ_LENGTH = 512  # Bengali text can be longer

# Training (optimized for L4 24GB with larger dataset)
NUM_EPOCHS = 3        # Larger dataset = fewer epochs needed
BATCH_SIZE = 16       # L4 can handle this
GRAD_ACCUM_STEPS = 4  # Effective batch = 64
LEARNING_RATE = 1e-4  # Slightly lower for stability
WARMUP_RATIO = 0.05   # Less warmup with more data

# LoRA (higher rank for 407 classes)
LORA_R = 64           # Higher rank for more classes
LORA_ALPHA = 128      # 2x rank
LORA_DROPOUT = 0.05
LORA_TARGET_MODULES = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]

# Seed
SEED = 42

# Instruction template (Bengali-aware)
INSTRUCTION_TEMPLATE = "Classify the intent of this Bengali customer query: {text}"

print(f"Model: {MODEL_NAME}")
print(f"Number of intents: {len(INTENT_TAGS)}")
print(f"Batch size: {BATCH_SIZE} x {GRAD_ACCUM_STEPS} = {BATCH_SIZE * GRAD_ACCUM_STEPS} effective")
print(f"LoRA rank: {LORA_R}")

Model: HuggingFaceTB/SmolLM2-135M
Number of intents: 407
Batch size: 16 x 4 = 64 effective
LoRA rank: 64


## 6. Prepare Dataset for SFT

In [7]:
from datasets import Dataset

def format_for_sft(row):
    """
    Format example for generative SFT.

    Input: Bengali question + tag
    Output: Formatted instruction-response pair
    """
    instruction = INSTRUCTION_TEMPLATE.format(text=row['question'])
    formatted = f"User: {instruction}\nAssistant: {row['tag']}"
    return {'text': formatted, 'intent': row['tag']}

# Convert DataFrames to formatted datasets
print("Formatting training data...")
train_formatted = [format_for_sft(row) for _, row in train_df.iterrows()]
train_dataset = Dataset.from_list(train_formatted)

print("Formatting evaluation data...")
eval_formatted = [format_for_sft(row) for _, row in eval_df.iterrows()]
eval_dataset = Dataset.from_list(eval_formatted)

print(f"\nTrain dataset: {len(train_dataset)} samples")
print(f"Eval dataset: {len(eval_dataset)} samples")

# Show formatted sample
print(f"\nFormatted sample:")
print(train_dataset[0]['text'])

Formatting training data...
Formatting evaluation data...

Train dataset: 78616 samples
Eval dataset: 11457 samples

Formatted sample:
User: Classify the intent of this Bengali customer query: "একাউন্ট লক করা হয়েছে" দেখাচ্ছে, সমাধান কী?
Assistant: account_locked


## 7. Load Model and Apply LoRA

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, TaskType, get_peft_model

# Load tokenizer
print(f"Loading tokenizer: {MODEL_NAME}")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

# Load model
print(f"Loading model: {MODEL_NAME}")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

# Enable gradient checkpointing
model.gradient_checkpointing_enable()

print(f"Model loaded. Parameters: {model.num_parameters():,}")

Loading tokenizer: HuggingFaceTB/SmolLM2-135M


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Model loaded. Parameters: 134,515,008


In [9]:
# Configure LoRA with higher rank for 407 classes
print("Configuring LoRA...")
lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=LORA_TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

# Apply LoRA
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

Configuring LoRA...
trainable params: 19,537,920 || all params: 154,052,928 || trainable%: 12.6826


## 8. Train

In [10]:
from trl import SFTTrainer, SFTConfig
from transformers import set_seed

# Set seed
set_seed(SEED)

# Configure training
training_args = SFTConfig(
    output_dir=OUTPUT_DIR,

    # Training schedule
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE * 2,
    gradient_accumulation_steps=GRAD_ACCUM_STEPS,

    # Optimizer
    learning_rate=LEARNING_RATE,
    weight_decay=0.01,
    warmup_ratio=WARMUP_RATIO,
    lr_scheduler_type="cosine",
    optim="adamw_8bit",

    # Mixed precision
    bf16=True,

    # Logging & saving
    logging_steps=50,
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=1000,
    save_total_limit=2,

    # Data
    max_length=MAX_SEQ_LENGTH,
    packing=False,

    # Other
    seed=SEED,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
)

# Initialize trainer
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    args=training_args,
)

print("Trainer initialized. Ready to train!")
print(f"Total training steps: {len(train_dataset) // (BATCH_SIZE * GRAD_ACCUM_STEPS) * NUM_EPOCHS}")

Adding EOS to train dataset:   0%|          | 0/78616 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/78616 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/78616 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/11457 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/11457 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/11457 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


Trainer initialized. Ready to train!
Total training steps: 3684


In [11]:
# Train!
print("Starting training...")
print(f"Dataset: ~78k Bengali NID queries, 407 intents")
print(f"This will take approximately 45-60 minutes on L4 GPU.")
print("-" * 50)

trainer.train()

print("-" * 50)
print("Training complete!")

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 0}.


Starting training...
Dataset: ~78k Bengali NID queries, 407 intents
This will take approximately 45-60 minutes on L4 GPU.
--------------------------------------------------


Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
500,0.558800,0.550443,0.612100,4134111.000000,0.830580
1000,0.476700,0.483418,0.537541,8264813.000000,0.848550
1500,0.440100,0.452769,0.494303,12388406.000000,0.857347
2000,0.419300,0.432177,0.479815,16523306.000000,0.863284
2500,0.396900,0.419513,0.454079,20649942.000000,0.867146
3000,0.391200,0.412402,0.448746,24783172.000000,0.869596
3500,0.389300,0.409406,0.447636,28914817.000000,0.870544


--------------------------------------------------
Training complete!


## 9. Save Model

In [12]:
# Save model to Google Drive
print(f"Saving model to {OUTPUT_DIR}...")
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

# Also save intent mappings
import json
with open(f"{OUTPUT_DIR}/intent_mappings.json", "w", encoding="utf-8") as f:
    json.dump({"id2intent": ID2INTENT, "intent2id": INTENT2ID}, f, ensure_ascii=False, indent=2)

print(f"Model saved successfully!")
print(f"Files in {OUTPUT_DIR}:")
!ls -la {OUTPUT_DIR}

Saving model to /content/drive/MyDrive/models/smollm2-bengali-nid-intent...
Model saved successfully!
Files in /content/drive/MyDrive/models/smollm2-bengali-nid-intent:
total 81114
-rw------- 1 root root     1058 Dec 20 07:35 adapter_config.json
-rw------- 1 root root 78207176 Dec 20 07:35 adapter_model.safetensors
drwx------ 2 root root     4096 Dec 20 07:14 checkpoint-3000
drwx------ 2 root root     4096 Dec 20 07:34 checkpoint-3687
-rw------- 1 root root    38936 Dec 20 07:35 intent_mappings.json
-rw------- 1 root root   466391 Dec 20 07:35 merges.txt
-rw------- 1 root root     1559 Dec 20 07:35 README.md
-rw------- 1 root root      863 Dec 20 07:35 special_tokens_map.json
-rw------- 1 root root     3720 Dec 20 07:35 tokenizer_config.json
-rw------- 1 root root  3522656 Dec 20 07:35 tokenizer.json
-rw------- 1 root root     6289 Dec 20 07:35 training_args.bin
-rw------- 1 root root   800662 Dec 20 07:35 vocab.json


## 10. Evaluate

In [13]:
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from collections import Counter

def extract_intent(response, intent_tags):
    """Extract intent from model response."""
    response = response.strip().lower()

    # Try exact match first
    for intent in intent_tags:
        if intent.lower() in response:
            return intent

    return None

def evaluate_model(model, tokenizer, eval_df, num_samples=None):
    """Evaluate model on test set."""
    model.eval()

    if num_samples:
        eval_df = eval_df.head(num_samples)

    predictions = []
    true_labels = []

    for _, row in tqdm(eval_df.iterrows(), total=len(eval_df), desc="Evaluating"):
        question = row['question']
        true_intent = row['tag']
        prompt = f"User: {INSTRUCTION_TEMPLATE.format(text=question)}\nAssistant:"

        # Tokenize
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=MAX_SEQ_LENGTH)
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        # Generate
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=64,
                do_sample=False,
                pad_token_id=tokenizer.pad_token_id,
            )

        # Decode
        response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
        predicted_intent = extract_intent(response, INTENT_TAGS)

        predictions.append(predicted_intent)
        true_labels.append(true_intent)

    return predictions, true_labels

# Evaluate on subset first (full eval takes time with 11k samples)
print("Evaluating on 2000 samples (for speed)...")
print("For full evaluation, change num_samples=None")
predictions, true_labels = evaluate_model(model, tokenizer, eval_df, num_samples=2000)

Evaluating on 2000 samples (for speed)...
For full evaluation, change num_samples=None


Evaluating:   0%|          | 0/2000 [00:00<?, ?it/s]

In [14]:
# Compute metrics
valid_mask = [p is not None for p in predictions]
valid_preds = [INTENT2ID.get(p, -1) for p in predictions]
valid_true = [INTENT2ID.get(t, -1) for t in true_labels]

# Filter valid
filtered_preds = [p for p, m in zip(valid_preds, valid_mask) if m and p != -1]
filtered_true = [t for t, m, p in zip(valid_true, valid_mask, valid_preds) if m and p != -1]

# Calculate metrics
if len(filtered_preds) > 0:
    accuracy = accuracy_score(filtered_true, filtered_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(
        filtered_true, filtered_preds, average="weighted", zero_division=0
    )
else:
    accuracy = precision = recall = f1 = 0.0

print("=" * 50)
print("EVALUATION RESULTS")
print("=" * 50)
print(f"Total samples: {len(predictions)}")
print(f"Valid predictions: {sum(valid_mask)} ({100*sum(valid_mask)/len(predictions):.1f}%)")
print(f"")
print(f"Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")
print("=" * 50)

# Show top confusions
print("\nTop 10 Confusions:")
confusions = [(t, p) for p, t in zip(predictions, true_labels) if p != t and p is not None]
for (true, pred), count in Counter(confusions).most_common(10):
    print(f"  {true} -> {pred}: {count}")

EVALUATION RESULTS
Total samples: 2000
Valid predictions: 1997 (99.8%)

Accuracy:  0.5383 (53.83%)
Precision: 0.6410
Recall:    0.5383
F1 Score:  0.5565

Top 10 Confusions:
  present_address_change_process_duration -> present_address_change_process: 13
  card_correction_fees_payment_reference_nid_number_query -> card_correction_fees: 13
  nid_wallet_download_alternative -> nid_wallet_download: 13
  account_locked_unlock_request -> account_locked: 10
  information_correct_but_account_locked -> account_locked: 10
  voter_area_change_online -> voter_area_change: 9
  permanent_address_change_process_duration -> permanent_address_change_process: 9
  voter_area_change_not_reflected -> voter_area_change: 7
  voter_area_change_ongoing -> voter_area_change: 6
  together_present_and_permanent_address_change -> migration_personal_attendance_required: 6


## 11. Interactive Inference

In [15]:
def classify_intent(query, model, tokenizer):
    """Classify intent for a single Bengali query."""
    prompt = f"User: {INSTRUCTION_TEMPLATE.format(text=query)}\nAssistant:"

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=MAX_SEQ_LENGTH)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=64,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
        )

    response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
    intent = extract_intent(response, INTENT_TAGS)

    return intent, response

# Get answer for intent from tag_answer_df
def get_answer(intent):
    """Get Bengali answer for an intent."""
    row = tag_answer_df[tag_answer_df['tag'] == intent]
    if len(row) > 0:
        return row.iloc[0]['answer']
    return "উত্তর পাওয়া যায়নি।"

# Test with sample Bengali queries
test_queries = [
    "আমার এনআইডি একাউন্ট লক হয়ে গেছে, কিভাবে আনলক করবো?",
    "কার্ড হারিয়ে গেলে কি করতে হবে?",
    "জাতীয় পরিচয়পত্রে নাম সংশোধন করতে চাই",
    "ভোটার আইডি কার্ডের ঠিকানা পরিবর্তন করতে কি কি লাগবে?",
    "স্মার্ট কার্ড কবে পাবো?",
]

print("Testing with Bengali queries:")
print("=" * 70)
for query in test_queries:
    intent, raw = classify_intent(query, model, tokenizer)
    answer = get_answer(intent) if intent else "Intent not recognized"
    print(f"Query: {query}")
    print(f"Intent: {intent}")
    print(f"Answer: {answer[:100]}..." if len(answer) > 100 else f"Answer: {answer}")
    print("-" * 70)

Testing with Bengali queries:
Query: আমার এনআইডি একাউন্ট লক হয়ে গেছে, কিভাবে আনলক করবো?
Intent: account_locked
Answer: ভুল তথ্য দিয়ে রেজিস্ট্রেশন বা লগইনের চেষ্টা করা হলে স্বয়ংক্রিয়ভাবে একাউন্টটি লক হয়ে যায়। অনুগ্রহপূর্...
----------------------------------------------------------------------
Query: কার্ড হারিয়ে গেলে কি করতে হবে?
Intent: card_lost_new_card_fee
Answer: এনআইডি অনলাইন পোর্টালে প্রবেশ করে ফিস ট্যাবের মাধ্যমে আপনার নির্দিষ্ট আবেদনের ফী এর পরিমাণ জেনে নিন।
----------------------------------------------------------------------
Query: জাতীয় পরিচয়পত্রে নাম সংশোধন করতে চাই
Intent: how_to_apply_for_correction_in_online_portal
Answer: প্রথমে services.nidw.gov.bd/ ওয়েবসাইট ভিজিট করে একাউন্ট রেজিস্ট্রেশন করুন এবং নির্ধারিত ফী প্রদান পূ...
----------------------------------------------------------------------
Query: ভোটার আইডি কার্ডের ঠিকানা পরিবর্তন করতে কি কি লাগবে?
Intent: address_change_online_new
Answer: হ্যাঁ, করা যাবে। তবে ঠিকানা পরিবর্তনের কারণে যদি ভোটার এলাকা পরিবর্তন হ

## 12. Push to HuggingFace Hub

In [17]:
from huggingface_hub import login
login()

In [19]:
# Push model to Hub
HF_REPO_NAME = "ehzawad/smollm2-bengali-nid-intent"

print(f"Pushing model to HuggingFace Hub: {HF_REPO_NAME}")

model.push_to_hub(HF_REPO_NAME)
tokenizer.push_to_hub(HF_REPO_NAME)

print(f"\nModel uploaded successfully!")
print(f"View at: https://huggingface.co/{HF_REPO_NAME}")

Pushing model to HuggingFace Hub: ehzawad/smollm2-bengali-nid-intent


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:   0%|          | 29.7kB / 78.2MB            

README.md: 0.00B [00:00, ?B/s]


Model uploaded successfully!
View at: https://huggingface.co/ehzawad/smollm2-bengali-nid-intent


## Done!

Your Bengali NID intent classification model is now:
- Saved to Google Drive at: `/content/drive/MyDrive/models/smollm2-bengali-nid-intent`
- Pushed to HuggingFace Hub

To load the model later:
```python
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

base_model = AutoModelForCausalLM.from_pretrained("HuggingFaceTB/SmolLM2-135M")
model = PeftModel.from_pretrained(base_model, "ehzawad/smollm2-bengali-nid-intent")
tokenizer = AutoTokenizer.from_pretrained("ehzawad/smollm2-bengali-nid-intent")
```